### Automacao web busca e retirada e alteracao em planilha

In [ ]:
#instalar selenim
#!pip install selenium

#baixar o webdrive de acordo com o navegador usado:
#chorme -> chormedriver
#firefox -> geckodriver
#colocar o executavel ".exe" na pasta onde está instalado o python "python.exe"

In [1]:
#documentacao:
#https://selenium-python.readthedocs.io/getting-started.html
from selenium import webdriver #controlar o navegador
from selenium.webdriver.common.keys import Keys #usar o teclado

#https://selenium-python.readthedocs.io/locating-elements.html
from selenium.webdriver.common.by import By #localizar itens no navegador

# para rodar o chrome em 2º plano
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True 
navegador = webdriver.Chrome(options=chrome_options)

#abrir o navegador
navegador = webdriver.Chrome()

#acessar o google
navegador.get("https://www.google.com/")

# Passo 1: Pegar a cotação do Dólar
#no google click com botao direito vá em "inspecionar" use a opcao "select an element inthe page to inspect" ou Ctrl + Shift+C
#depois de selecionar clica na opcao copy XPATH

#para digitar na pesquisa do google
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotacao dólar')
#para clicar enter na pesquisa
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

#para digitar e clicar enter
#navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotacao dólar', Kyes.ENTER)

#pegar (get) o atributo valor da cotacao
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_dolar)

#cotacao euro:
#retorna ao google:
navegador.get("https://www.google.com/")

#para digitar e clicar enter:
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotacao euro', Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

#cotacao ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value')

cotacao_ouro = cotacao_ouro.replace(',','.')

print(cotacao_ouro)

#fechar o navegador
navegador.quit()

5.122599999999999
5.621208271
321.93


### Atualiza a base de preços com as novas cotações

In [2]:
import pandas as pd
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [3]:
#atualizar as cotacoes

#.loc:
#tabela.loc[linhas, colunas]=cotacao_dolar
#"tabela, localiza as linhas que são dolar e atualiza o valor da coluna cotacao:"
#atualizar "cotacao" nas linhas onde a coluna "moeda" = Dólar:
tabela.loc[tabela["Moeda"]=="Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"]== 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela["Moeda"]== 'Ouro', 'Cotação'] = float(cotacao_ouro)

#atualizar colunas "Preço de Compra" e 'Preço de Venda'
tabela["Preço de Compra"] = tabela["Preço Original"]* tabela["Cotação"]
tabela['Preço de Venda'] = tabela['Preço de Compra']*tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.122600,5122.548774,1.40,7171.568284
1,Carro Renault,4500.00,Euro,5.621208,25295.437220,2.00,50590.874439
2,Notebook Dell,899.99,Dólar,5.122600,4610.288774,1.70,7837.490916
3,IPhone,799.00,Dólar,5.122600,4092.957400,1.70,6958.027580
4,Carro Fiat,3000.00,Euro,5.621208,16863.624813,1.90,32040.887145
5,Celular Xiaomi,480.48,Dólar,5.122600,2461.306848,2.00,4922.613696
6,Joia 20g,20.00,Ouro,321.930000,6438.600000,1.15,7404.390000


### Exportar a nova base de preços atualizada

In [4]:
tabela.to_excel('Produtos_Atualizados.xlsx', index= False)